# After patient data has been queried, we are moving to model practise

## Reference

* https://github.com/YerevaNN/mimic3-benchmarks

## Citations

> Johnson, Alistair EW, David J. Stone, Leo A. Celi, and Tom J. Pollard. "The MIMIC Code Repository: enabling reproducibility in critical care research." Journal of the American Medical Informatics Association (2017): ocx084.
* Github: https://github.com/MIT-LCP/mimic-code
* Zenodo: https://doi.org/10.5281/zenodo.821872
* Structure tested: multitask RNN architecture: https://arxiv.org/abs/1703.07771 

* Hrayr Harutyunyan, Hrant Khachatrian, David C. Kale, and Aram Galstyan. Multitask Learning and Benchmarking with Clinical Time Series Data. arXiv:1703.07771: https://arxiv.org/abs/1703.07771
<br>

* Mimic3 benchmarks for machine learning: https://github.com/YerevaNN/mimic3-benchmarks
    1. early triage and risk assessment, i.e., mortality prediction
    2. prediction of physiologic decompensation
    3. identification of high cost patients, i.e. length of stay forecasting
    4. characterization of complex, multi-system diseases, i.e., acute care phenotyping

## Tools used

In [151]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import sklearn                              # generic machine learning tool kit
import keras                                # for LSTM model to handle time series

## Getting data prepared for ML

Note: The whole process might take a few hours...

### Training and Testing

1\. Clone the repo. Following code will create ~/data folder to contain ML train/test/val

`git clone https://github.com/YerevaNN/mimic3-benchmarks/
cd mimic3-benchmarks/`

2\. The following command takes MIMIC-III CSVs, the steps takes around ~3 hour.
- generates one directory per `SUBJECT_ID`, totally 33798 folders unnder ~/data/root/
- writes ICU stay information to `data/{SUBJECT_ID}/stays.csv`
- diagnoses to `data/{SUBJECT_ID}/diagnoses.csv`
- and events to `data/{SUBJECT_ID}/events.csv`. 

`python -m mimic3benchmark.scripts.extract_subjects {PATH TO MIMIC-III CSVs} data/root/`

3\. The following command attempts to fix some issues (ICU stay ID is missing) and removes the events that have missing information. About 80% of events remain after removing all suspicious rows.

`python -m mimic3benchmark.scripts.validate_events data/root/`

4\. The next command breaks up per-subject data into separate episodes (pertaining to ICU stays). 
- Time series of events are stored in `{SUBJECT_ID}/episode{#}_timeseries.csv` (where # counts distinct episodes) 
- episode-level information (patient age, gender, ethnicity, height, weight) and outcomes (mortality, length of stay, diagnoses) are stores in `{SUBJECT_ID}/episode{#}.csv`. 
- This script requires two files, one that maps event `ITEMIDs` to clinical variables and another that defines valid ranges for clinical variables (for detecting outliers, etc.). 
- Outlier detection is disabled in the current version.

`python -m mimic3benchmark.scripts.extract_episodes_from_subjects data/root/`

5\. The next command splits the whole dataset into training and testing sets. Note that the train/test split is the same of all tasks.

`python -m mimic3benchmark.scripts.split_train_and_test data/root/`

6\. The following commands will generate task-specific datasets, which can later be used in models. These commands are independent, if you are going to work only on one benchmark task, you can run only the corresponding command.


1. early triage and risk assessment, i.e., mortality prediction <br>
`python -m mimic3benchmark.scripts.create_in_hospital_mortality data/root/ data/in-hospital-mortality/`
<br>
    
2. prediction of physiologic decompensation<br>
`python -m mimic3benchmark.scripts.create_decompensation data/root/ data/decompensation/`
<br>

3. identification of high cost patients, i.e. length of stay forecasting<br>
`python -m mimic3benchmark.scripts.create_length_of_stay data/root/ data/length-of-stay/`
<br>

4. characterization of complex, multi-system diseases, i.e., acute care phenotyping<br>
`python -m mimic3benchmark.scripts.create_phenotyping data/root/ data/phenotyping/`<br>
`python -m mimic3benchmark.scripts.create_multitask data/root/ data/multitask/`





7\. After the above commands are done, there will be a directory data/{task} for each created benchmark task. These directories have two sub-directories: `train` and `test`. Each of them contains bunch of ICU stays and one file with name `listfile.csv`, which lists all samples in that particular set. 

Each row of listfile.csv has the following form: `icu_stay`, `period_length`, `label(s)`. A row specifies a sample for which the input is the collection of ICU event of `icu_stay` that occurred in the first `period_length` hours of the stay and the target is/are label(s). 

In in-hospital mortality prediction task `period_length` is always 48 hours, so it is not listed in corresponding listfiles.

8\. Seems some record missing for mortality data<br>
Note remaining tasks also have data missing, won't copy all details.

`(PY36) $ python -m mimic3benchmark.scripts.create_in_hospital_mortality data/root/ data/in-hospital-mortality/
processed 5000 / 5070 patients
 3236
(length of stay is missing) 10128 episode1_timeseries.csv
processed 100 / 28728 patients
(length of stay is missing) 10168 episode1_timeseries.csv
processed 2400 / 28728 patients
(no events in ICU)  14219 episode1_timeseries.csv
processed 2600 / 28728 patients
(no events in ICU)  14469 episode1_timeseries.csv
processed 4700 / 28728 patients
(no events in ICU)  18350 episode1_timeseries.csv
processed 5200 / 28728 patients
(no events in ICU)  19097 episode1_timeseries.csv
processed 5600 / 28728 patients
(no events in ICU)  19872 episode1_timeseries.csv
processed 16100 / 28728 patients
(length of stay is missing) 499 episode1_timeseries.csv
processed 28700 / 28728 patients
 17903`


After these operations, `mimic3-benchmarks` folder will include below sub-folders, and every sub-folder will include datasets for `test` and `train` :
    
`decompensation` <br>
`in-hospital-mortality` <br>
`length-of-stay` <br>
`multitask` <br>
`phenotyping` <br>
`root` <br>

### Validation

We want to provide Train / Validation split before running the model:

`python -m mimic3models.split_train_val {dataset-directory}`

The `{dataset-directory}` can be either:

`data/in-hospital-mortality
data/decompensation
data/length-of-stay
data/phenotyping
data/multitask`

## Baseline Model Options

For each of the four main tasks:
    1. Early triage and risk assessment, i.e., mortality prediction
    2. Prediction of physiologic decompensation
    3. Identification of high cost patients, i.e. length of stay forecasting
    4. Characterization of complex, multi-system diseases, i.e., acute care phenotyping
    
    
7 baselines are provided:<br>
Note: Baseline models could take longer time because it's using grid search in pipeline.  

    1. Linear/logistic regression
    2. Standard LSTM
    3. Standard LSTM + deep supervision
    4. Channel-wise LSTM
    5. Channel-wise LSTM + deep supervision
    6. Multitask standard LSTM
    7. Multitask channel-wise LSTM

##  Logistic Regression Model test (In-hospital Mortality Prediction)
Note: This would be a typical ML use case so chose to practise.

`python -um mimic3models.in_hospital_mortality.logistic.main --l2 --C 0.001 --output_dir mimic3models/in_hospital_mortality/logistic`

<b>Break down the model into Jupyter Notebook commands for better understanding</b>

In [143]:
# add mimic3_benchmarks into system path so can import modules in its subdirectory
# you can check system path by sys.path
import sys
from os.path import abspath, join, dirname
sys.path.insert(0, join(os.path.abspath('./'), 'mimic3_benchmarks'))

# import libraries
from sklearn.preprocessing import Imputer, StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV
import argparse
import json

# import modules
from mimic3_benchmarks.mimic3benchmark.readers import InHospitalMortalityReader
from mimic3_benchmarks.mimic3models import common_utils
from mimic3_benchmarks.mimic3models.metrics import print_metrics_binary
from mimic3_benchmarks.mimic3models.in_hospital_mortality.utils import save_results

In [73]:
# define read feature and extract feature
def read_and_extract_features(reader, period, features):
    ret = common_utils.read_chunk(reader, reader.get_number_of_examples())
    # ret = common_utils.read_chunk(reader, 100)
    X = common_utils.extract_features_from_rawdata(ret['X'], ret['header'], period, features)
    return (X, ret['y'], ret['name'])

In [142]:
# use arguments to parse the arments for command line running
# we also can use default values to run in Jupyter Notebook
# or use Grid search to targetly choose some parameters
parser = argparse.ArgumentParser()
parser.add_argument('--C', type=float, default=1.0, help='inverse of L1 / L2 regularization')
parser.add_argument('--l1', dest='l2', action='store_false')
parser.add_argument('--l2', dest='l2', action='store_true')
parser.set_defaults(l2=True)

parser.add_argument('--period', type=str, default='all', 
                    help='specifies which period extract features from',
                    choices=['first4days', 'first8days', 'last12hours', 'first25percent', 'first50percent', 'all'])
parser.add_argument('--features', type=str, default='all', 
                    help='specifies what features to extract',
                    choices=['all', 'len', 'all_but_len'])
parser.add_argument('--data', type=str, 
                    help='Path to the data of in-hospital mortality task',
                    default=os.path.join(os.path.abspath('./'), 'mimic3_benchmarks/data/in-hospital-mortality/'))
parser.add_argument('--output_dir', type=str, 
                    help='Directory relative which all output files are stored',
                    default='.')
args = parser.parse_args(args=[])

print('type(args): ',type(args))
print(args.C)
print(args.data)
print(args.features)
print(args.l2)
print(args.output_dir)
print(args.period)

type(args):  <class 'argparse.Namespace'>
1.0
C:\Users\ericx\Jupyter Projects\MGH_medical_AI\mimic3_benchmarks/data/in-hospital-mortality/
all
True
.
all


In [101]:
# read data
train_reader = InHospitalMortalityReader(dataset_dir=os.path.join(args.data, 'train'),
                                             listfile=os.path.join(args.data, 'train_listfile.csv'),
                                             period_length=48.0)

val_reader = InHospitalMortalityReader(dataset_dir=os.path.join(args.data, 'train'),
                                           listfile=os.path.join(args.data, 'val_listfile.csv'),
                                           period_length=48.0)

test_reader = InHospitalMortalityReader(dataset_dir=os.path.join(args.data, 'test'),
                                            listfile=os.path.join(args.data, 'test_listfile.csv'),
                                            period_length=48.0)

Reading data and extracting features ...


In [103]:
# extracting features, this is a longer process
(train_X, train_y, train_names) = read_and_extract_features(train_reader, args.period, args.features)
(val_X, val_y, val_names) = read_and_extract_features(val_reader, args.period, args.features)
(test_X, test_y, test_names) = read_and_extract_features(test_reader, args.period, args.features)
print('  train data shape = {}'.format(train_X.shape))
print('  validation data shape = {}'.format(val_X.shape))
print('  test data shape = {}'.format(test_X.shape))

  train data shape = (14681, 714)
  validation data shape = (3222, 714)
  test data shape = (3236, 714)


In [126]:
# data compensation (for missing values)
imputer = Imputer(missing_values=np.nan, strategy='mean', axis=0, verbose=0, copy=True)
imputer.fit(train_X)
train_X = np.array(imputer.transform(train_X), dtype=np.float32)
val_X = np.array(imputer.transform(val_X), dtype=np.float32)
test_X = np.array(imputer.transform(test_X), dtype=np.float32)

# data normalization
scaler = StandardScaler()
scaler.fit(train_X)
train_X = scaler.transform(train_X)
val_X = scaler.transform(val_X)
test_X = scaler.transform(test_X)

C:\Users\ericx\AppData\Local\conda\conda\envs\PY36\lib\site-packages\sklearn\utils\deprecation.py:58: DeprecationWarning: Class Imputer is deprecated; Imputer was deprecated in version 0.20 and will be removed in 0.22. Import impute.SimpleImputer from sklearn instead.
  warnings.warn(msg, category=DeprecationWarning)


In [138]:
# define parameters
# note add 'lr' to store results under 'lr' folder to differ other model results
penalty = ('l2' if args.l2 else 'l1')
file_name = '{}.{}.{}.{}.C{}'.format('lr',args.period, args.features, penalty, args.C)

Original paramters from the article

`LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=42, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)`

In [152]:
# train the model, grid search can be time consuming
# note article author state best model is under C = 0.001
# This was the orginal training, used args.C for the C value
    # logreg = LogisticRegression(penalty=penalty, C=args.C, random_state=42)
    # logreg.fit(train_X, train_y)
# Now, we will use Grid-search to test the best jypter parameters, instead of inputting by command line or default args
grid={"C":np.logspace(-3,0,4), "penalty":["l1", "l2"]}

logr = LogisticRegression()
logreg = GridSearchCV(logr, grid, cv = 10)
logreg.fit(train_X, train_y)

GridSearchCV(cv=10, error_score='raise-deprecating',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False),
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'C': array([0.001, 0.01 , 0.1  , 1.   ]), 'penalty': ['l1', 'l2']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [168]:
# found best parameter combination is C=0.1 and penalty=l1
# which differs from C=0.001 and panalty=l2 as article used, but we just observe here since difference are not that much in linear model
print("tuned hpyerparameters :(best parameters) ",logreg.best_params_)
print("accuracy :",logreg.best_score_)

tuned hpyerparameters :(best parameters)  {'C': 0.1, 'penalty': 'l1'}
accuracy : 0.8806620802397657


In [169]:
# creating new directory to store results, under ./result/In_hospital_Mortality
result_dir = os.path.join(args.output_dir, 'results/In_hospital_Mortality')
common_utils.create_directory(result_dir)
print('results saved to:', result_dir, '\n')

# predictions
# output of logreg.predict_proba is a N x 2 matrix
# output[:, 0] means the probability of prediction result 0 (alive)
# output[:, 0] means the probability of prediction result 1 (dead)

print('The training data sets----------------')
prediction_train = logreg.predict_proba(train_X)
with open(os.path.join(result_dir, 'lr_train_{}.json'.format(file_name)), 'w') as res_file:
    ret = print_metrics_binary(train_y, prediction_train)
    ret = {k : float(v) for k, v in ret.items()}
    json.dump(ret, res_file)
print('\n')

print('The validation data sets----------------')
prediction_val = logreg.predict_proba(val_X)
with open(os.path.join(result_dir, 'lr_val_{}.json'.format(file_name)), 'w') as res_file:
    ret = print_metrics_binary(val_y, prediction_val)
    ret = {k: float(v) for k, v in ret.items()}
    json.dump(ret, res_file)
print('\n')

print('The testing data sets----------------')
prediction_test = logreg.predict_proba(test_X)[:, 1]
with open(os.path.join(result_dir, 'lr_test_{}.json'.format(file_name)), 'w') as res_file:
    ret = print_metrics_binary(test_y, prediction_test)
    ret = {k: float(v) for k, v in ret.items()}
    json.dump(ret, res_file)
print('\n')

save_results(test_names, prediction, test_y, 
             os.path.join(args.output_dir, 'predictions/In_hospital_Mortality', file_name + '.csv'))

results saved to: .\results/In_hospital_Mortality 

The training data sets----------------
confusion matrix:
[[12395   299]
 [ 1349   638]]
accuracy = 0.8877460956573486
precision class 0 = 0.9018480777740479
precision class 1 = 0.6808964610099792
recall class 0 = 0.9764455556869507
recall class 1 = 0.3210870623588562
AUC of ROC = 0.8711970093301433
AUC of PRC = 0.5690212947090911
min(+P, Se) = 0.539738430583501


The validation data sets----------------
confusion matrix:
[[2701   85]
 [ 304  132]]
accuracy = 0.8792675137519836
precision class 0 = 0.898835301399231
precision class 1 = 0.6082949042320251
recall class 0 = 0.9694902896881104
recall class 1 = 0.302752286195755
AUC of ROC = 0.8424099527783087
AUC of PRC = 0.4926910001777761
min(+P, Se) = 0.5114678899082569


The testing data sets----------------
confusion matrix:
[[2785   77]
 [ 247  127]]
accuracy = 0.8998764157295227
precision class 0 = 0.9185356497764587
precision class 1 = 0.6225489974021912
recall class 0 = 0.973095715

Note: This is just a trial, as we know the normal linear model won't have good performance on time series data. LSTM will give much better results.